In [15]:
# =========================================================
# SEÇÃO 1: IMPORTAÇÕES E SETUP GERAL
# =========================================================

In [16]:
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import warnings
import itertools

# Libs de Modelagem
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.datasets import get_rdataset
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, MLP, LSTM, Autoformer

# Libs de Avaliação
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tqdm import tqdm
from IPython.display import display

warnings.filterwarnings("ignore")

In [17]:
# =========================================================
# SEÇÃO 2: FUNÇÕES AUXILIARES (SETUP E PROCESSAMENTO)
# =========================================================

In [18]:
def definir_seed(seed_value=42):
    np.random.seed(seed_value)
    random.seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

In [ ]:
def salvar_dataset(serie, dataset_name):
    dir_path = "./datasets/"
    os.makedirs(dir_path, exist_ok=True)
    file_path = os.path.join(dir_path, f"{dataset_name.lower()}.csv")
    df = pd.DataFrame({"date": serie.index, "value": serie.values})
    df.to_csv(file_path, index=False)
    print(f"-> Cópia do dataset '{dataset_name}' salva em: {file_path}")

def carregar_serie(nome):
    print(f"Buscando dados de '{nome}' via statsmodels...")
    nome_base = nome.lower()

    if nome_base == "airpassengers":
        df = get_rdataset("AirPassengers", package="datasets").data
        serie = pd.Series(df['value'].values, index=pd.date_range(start="1949-01-01", periods=len(df), freq="MS"),
                          name="AirPassengers")
    elif nome_base == "lynx":
        df = get_rdataset("lynx", package="datasets").data
        serie = pd.Series(df['value'].values, index=pd.date_range(start="1821", periods=len(df), freq="A"), name="Lynx")
    elif nome_base == "co2":
        df = get_rdataset("CO2", package="datasets").data
        df = df.ffill()
        serie = pd.Series(df['value'].values, index=pd.date_range(start="1958-03-29", periods=len(df), freq="MS"),
                          name="CO2")
    elif nome_base == "sunspots":
        df = get_rdataset("sunspots", package="datasets").data
        serie = pd.Series(df['value'].values, index=pd.date_range(start="1749-01-01", periods=len(df), freq="MS"),
                          name="Sunspots")
    elif nome_base == "austres":
        df = get_rdataset("austres", package="datasets").data
        serie = pd.Series(df['value'].values, index=pd.date_range(start="1971-03-01", periods=len(df), freq="QS-MAR"),
                          name="AustralianResidents")
    elif nome_base == "nottem":
        df = get_rdataset("nottem", package="datasets").data
        serie = pd.Series(df['value'].values, index=pd.date_range(start="1920-01-01", periods=len(df), freq="MS"),
                          name="Nottingham")
    else:
        raise ValueError(f"Série '{nome}' não reconhecida.")

    salvar_dataset(serie, nome)
    return serie

In [20]:
def dividir_serie_temporal(serie, percentual_treino=0.7, percentual_validacao=0.15):
    tamanho_total = len(serie)
    if tamanho_total < 20: 
        percentual_treino=0.8
        percentual_validacao=0.0
    ponto_corte_treino = int(tamanho_total * percentual_treino)
    ponto_corte_validacao = int(tamanho_total * (percentual_treino + percentual_validacao))
    treino = serie.iloc[:ponto_corte_treino]
    validacao = serie.iloc[ponto_corte_treino:ponto_corte_validacao]
    teste = serie.iloc[ponto_corte_validacao:]
    return treino, validacao, teste

def preparar_dados_para_neuralforecast(serie, nome_serie):
    df = serie.reset_index()
    df.columns = ['ds', 'y']
    df['unique_id'] = nome_serie
    return df

In [21]:
# =========================================================
# SEÇÃO 3: FUNÇÕES PARA CÁLCULO DE MÉTRICAS E MODELAGEM
# =========================================================

In [22]:
def calcular_metricas(y_true, y_pred, y_train):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100 if np.all(y_true != 0) else np.inf
    n = len(y_train)
    d = np.sum(np.abs(y_train[1:] - y_train[:-1])) / (n - 1) if n > 1 else np.nan
    mase = np.mean(np.abs(y_true - y_pred)) / d if d is not np.nan and d > 0 else np.inf
    return {'RMSE': rmse, 'MAPE(%)': mape, 'MASE': mase}

In [23]:
# =========================================================
# SEÇÃO 4: PIPELINE DE EXPERIMENTO COM TREINO-VALIDAÇÃO-TESTE
# =========================================================

In [24]:
def executar_experimento(nome_da_serie):
    """
    Executa o pipeline completo com metodologia Treino-Validação-Teste para todos os modelos.
    """
    try:
        # --- Configurações ---
        ORDEM_ARIMA_FIXA = (3, 1, 2)
        SEED = 42
        MAX_INPUT_SIZE = 50
        MAX_STEPS_NEURAL = 100

        definir_seed(SEED)
        
        # --- Preparação dos Dados ---
        serie_completa = carregar_serie(nome_da_serie)
        treino, validacao, teste = dividir_serie_temporal(serie_completa)
        
        if len(teste) < 2 or len(validacao) < 2: 
            print(f"AVISO: Dataset '{nome_da_serie}' muito pequeno para a divisão. Pulando.")
            return None, None

        freq = serie_completa.index.freqstr or pd.infer_freq(serie_completa.index)
        if freq is None: 
            print(f"ERRO: Frequência não determinada para '{nome_da_serie}'.")
            return None, None

        # Prepara os dados de treino uma vez para os modelos neurais
        df_treino_nf = preparar_dados_para_neuralforecast(treino, nome_da_serie)
        
        # Listas para guardar os resultados de cada etapa
        resultados_validacao = []
        resultados_teste = []

        # --- 1. Modelo ARIMA ---
        try:
            print("\nProcessando: ARIMA")
            modelo_arima = ARIMA(treino.asfreq(freq), order=ORDEM_ARIMA_FIXA).fit()
            # Prevê ambos os períodos: validação e teste
            preds_futuro_arima = modelo_arima.predict(start=validacao.index[0], end=teste.index[-1])
            preds_valid_arima = preds_futuro_arima[:len(validacao)]
            preds_teste_arima = preds_futuro_arima[-len(teste):]

            # Avalia na validação e no teste
            metricas_val = calcular_metricas(validacao.values, preds_valid_arima.values, treino.values); metricas_val['modelo'] = 'ARIMA'; resultados_validacao.append(metricas_val)
            metricas_test = calcular_metricas(teste.values, preds_teste_arima.values, treino.values); metricas_test['modelo'] = 'ARIMA'; resultados_teste.append(metricas_test)
        except Exception as e:
            print(f"AVISO: Modelo ARIMA falhou: {e}")

        # --- 2. Modelos Neurais Puros ---
        horizonte_total = len(validacao) + len(teste)
        input_size = min(2 * horizonte_total, MAX_INPUT_SIZE)
        modelos_para_testar = {'N-BEATS': NBEATS, 'MLP': MLP, 'LSTM': LSTM, 'Autoformer': Autoformer}

        for nome_modelo, classe_modelo in modelos_para_testar.items():
            try:
                print(f"Processando: {nome_modelo}")
                modelo_neural = [classe_modelo(input_size=input_size, h=horizonte_total, max_steps=MAX_STEPS_NEURAL, scaler_type='standard', random_seed=SEED)]
                nf = NeuralForecast(models=modelo_neural, freq=freq)
                nf.fit(df=df_treino_nf)
                preds_futuro_df = nf.predict()
                
                # Separa as previsões para validação e teste
                preds_valid_neural = preds_futuro_df[nome_modelo].values[:len(validacao)]
                preds_teste_neural = preds_futuro_df[nome_modelo].values[-len(teste):]

                # Avalia em ambas as etapas
                metricas_val = calcular_metricas(validacao.values, preds_valid_neural, treino.values); metricas_val['modelo'] = f'{nome_modelo} (MIMO)'; resultados_validacao.append(metricas_val)
                metricas_test = calcular_metricas(teste.values, preds_teste_neural, treino.values); metricas_test['modelo'] = f'{nome_modelo} (MIMO)'; resultados_teste.append(metricas_test)
            except Exception as e:
                print(f"AVISO: Modelo {nome_modelo} falhou: {e}")

        # --- 3. Modelo Híbrido ---
        if 'modelo_arima' in locals():
            try:
                print("Processando: Modelo Híbrido (HyS-MF)")
                residuos_treino = modelo_arima.resid
                df_residuos_treino_nf = preparar_dados_para_neuralforecast(residuos_treino, "residuos")
                modelo_residuos = [NBEATS(input_size=input_size, h=horizonte_total, max_steps=MAX_STEPS_NEURAL, scaler_type='standard', random_seed=SEED)]
                nf_residuos = NeuralForecast(models=modelo_residuos, freq=freq)
                nf_residuos.fit(df=df_residuos_treino_nf)
                preds_residuos_futuro = nf_residuos.predict()['NBEATS'].values
                
                # Separa as previsões de resíduos
                preds_residuos_valid = preds_residuos_futuro[:len(validacao)]
                preds_residuos_teste = preds_residuos_futuro[-len(teste):]

                # Previsão híbrida para validação e teste
                preds_hibrido_valid = preds_valid_arima.values + preds_residuos_valid
                preds_hibrido_teste = preds_teste_arima.values + preds_residuos_teste

                # Avalia em ambas as etapas
                metricas_val = calcular_metricas(validacao.values, preds_hibrido_valid, treino.values); metricas_val['modelo'] = 'Híbrido (HyS-MF)'; resultados_validacao.append(metricas_val)
                metricas_test = calcular_metricas(teste.values, preds_hibrido_teste, treino.values); metricas_test['modelo'] = 'Híbrido (HyS-MF)'; resultados_teste.append(metricas_test)
            except Exception as e:
                print(f"AVISO: Modelo Híbrido falhou: {e}")

        return pd.DataFrame(resultados_validacao), pd.DataFrame(resultados_teste)

    except Exception as e:
        print(f"ERRO GERAL no processamento do dataset '{nome_da_serie}': {e}")
        return None, None

In [25]:
# =========================================================
# SEÇÃO 4: ORQUESTRADOR E RELATÓRIO FINAL
# =========================================================

In [ ]:
LISTA_DE_DATASETS = ['AirPassengers', 'co2', 'lynx'] # Usando uma lista menor para exemplo
resultados_validacao_geral = []
resultados_teste_geral = []

for dataset in tqdm(LISTA_DE_DATASETS, desc="Processando todos os datasets"):
    df_validacao, df_teste = executar_experimento(dataset)
    
    if df_validacao is not None and not df_validacao.empty:
        df_validacao['dataset'] = dataset
        resultados_validacao_geral.append(df_validacao)
        
    if df_teste is not None and not df_teste.empty:
        df_teste['dataset'] = dataset
        resultados_teste_geral.append(df_teste)

# --- Salvando os resultados em arquivos CSV ---
if resultados_validacao_geral:
    df_validacao_final = pd.concat(resultados_validacao_geral)
    df_validacao_final.to_csv("./datasets/silver/resultados_validacao.csv", index=False)
    print("\nArquivo 'resultados_validacao.csv' salvo com sucesso!")

if resultados_teste_geral:
    df_teste_final = pd.concat(resultados_teste_geral)
    df_teste_final.to_csv("./datasets/silver/resultados_teste.csv", index=False)
    print("Arquivo 'resultados_teste.csv' salvo com sucesso!")

Processando todos os datasets:   0%|          | 0/3 [00:00<?, ?it/s]

Buscando dados de 'AirPassengers' via statsmodels...


Seed set to 42


-> Cópia do dataset 'AirPassengers' salva em: ./datasets\airpassengers.csv

Processando: ARIMA
Processando: N-BEATS


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 2.6 M  | train
-------------------------------------------------------
2.6 M     Trainable params
8.4 K     Non-trainable params
2.6 M     Total params
10.361    Total estimated model params size (MB)
31        Modules in train mode
0         Modules in eval mode


...

Epoch 99: 100%|██████████| 1/1 [00:00<00:00,  3.37it/s, v_num=1511, train_loss_step=0.0173, train_loss_epoch=0.0173]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 133.57it/s]

Processando todos os datasets: 100%|██████████| 3/3 [1:54:10<00:00, 2283.55s/it]



Arquivo 'resultados_validacao.csv' salvo com sucesso!
Arquivo 'resultados_teste.csv' salvo com sucesso!


In [27]:
# =================================================================================
# SEÇÃO 5: GERAÇÃO DE RELATÓRIOS A PARTIR DOS ARQUIVOS SALVOS
# =================================================================================

In [28]:
print("\n\n" + "="*60)
print("     GERANDO RELATÓRIOS A PARTIR DOS DADOS SALVOS")
print("="*60)

try:
    # Carrega os resultados do teste a partir do arquivo
    df_final_comparativo = pd.read_csv("resultados_teste.csv").rename(columns={'modelo': 'Modelo'})
    
    # Renomeia as colunas para o relatório
    rename_dict = {'RMSE': 'Mean RMSE', 'MAPE(%)': 'Mean MAPE(%)', 'MASE': 'Mean MASE'}
    df_final_comparativo.rename(columns=rename_dict, inplace=True)
    
    # --- Relatório 1: Tabela de Desempenho (baseado nos dados de TESTE) ---
    print("\n--- RELATÓRIO 1: DESEMPENHO FINAL (TESTE) POR MÉTRICA ---")

    def destacar_melhor_por_grupo(df):
        df_style = pd.DataFrame('', index=df.index, columns=df.columns)
        estilo_melhor = 'background-color: #4285F4; color: white; font-weight: bold;'
        for metrica in ['Mean RMSE', 'Mean MAPE(%)', 'Mean MASE']:
            if metrica in df.columns:
                idx_minimos = df.groupby('dataset')[metrica].idxmin()
                df_style.loc[idx_minimos, metrica] = estilo_melhor
        return df_style

    df_reporte_1 = df_final_comparativo.set_index(['dataset', 'Modelo'])
    
    styled_report_1 = (df_reporte_1.style
                       .format('{:.3f}')
                       .set_caption("O destaque indica o melhor modelo para cada métrica no conjunto de teste.")
                       .apply(destacar_melhor_por_grupo, axis=None))
    
    display(styled_report_1)

    # --- Relatório 2: Tabela de Ranking (baseado nos dados de TESTE) ---
    print("\n--- RELATÓRIO 2: RANKING DOS MODELOS (BASEADO EM RMSE DE TESTE) ---")
    
    df_final_comparativo['Rank'] = df_final_comparativo.groupby('dataset')['Mean RMSE'].rank().astype(int)
    df_pivot_rank = df_final_comparativo.pivot(index='dataset', columns='Modelo', values='Rank')
    df_pivot_rank.loc['Média do Rank'] = df_pivot_rank.mean(axis=0)

    styled_report_2 = (df_pivot_rank.style
                       .format('{:.1f}')
                       .highlight_min(axis=1, props='background-color: #4285F4; color: white; font-weight: bold;')
                       .set_caption("Ranking dos modelos (1 = Melhor, ...)."))
    
    display(styled_report_2)
    
except FileNotFoundError:
    print("\nERRO: Arquivo 'resultados_teste.csv' não encontrado.")
    print("Por favor, execute a Seção 4 (Etapa de Execução e Salvamento) primeiro para gerar os resultados.")



     GERANDO RELATÓRIOS A PARTIR DOS DADOS SALVOS

--- RELATÓRIO 1: DESEMPENHO FINAL (TESTE) POR MÉTRICA ---



--- RELATÓRIO 2: RANKING DOS MODELOS (BASEADO EM RMSE DE TESTE) ---


Modelo,ARIMA,Autoformer (MIMO),Híbrido (HyS-MF),LSTM (MIMO),MLP (MIMO)
dataset,,,,,
AirPassengers,5.0,1.0,4.0,3.0,2.0
co2,3.0,4.0,2.0,5.0,1.0
lynx,3.0,5.0,4.0,2.0,1.0
Média do Rank,3.7,3.3,3.3,3.3,1.3
